# 1. Pre- processing

In [ ]:
from __future__ import print_function
from google.colab import files 

import io
import pandas as pd


  
#Code to upload the dataset from your computer as a dataframe
df=pd.read_csv("/content/final_df.csv")


In [ ]:
#Code to take first 10,000 instances of the dataset.
df = df.iloc[:10000, :]
df.shape

(10000, 4)

In [ ]:
df.head()

,Unnamed: 0,product,desc,product_id
0,0,Strap top,solid jersey basic basics everyday womens shou...,1
1,1,Strap top (1),stripe jersey basic basics everyday womens sho...,2
2,2,OP T-shirt (Idro),solid clean lingerie womens lingerie visible m...,3
3,3,20 den 1p Stockings,solid tights basic tigh socks nightwear womens...,4
4,4,Shape Up 30 den 1p Tights,solid tights basic tigh socks nightwear womens...,5


In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#Tokenization of data
df['desc'] = df.apply(lambda row: nltk.word_tokenize(row['desc']), axis=1)

In [ ]:
#Lemmatization
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatizer(text):
  lemm = [wordnet_lemmatizer.lemmatize(word) for word in text]
  lemm = ' '.join(lemm)
  return lemm
  
df['desc']=df['desc'].apply(lambda x:lemmatizer(x))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df.head(10)

,Unnamed: 0,product,desc,product_id
0,0,Strap top,solid jersey basic basic everyday woman should...,1
1,1,Strap top (1),stripe jersey basic basic everyday woman shoul...,2
2,2,OP T-shirt (Idro),solid clean lingerie woman lingerie visible mi...,3
3,3,20 den 1p Stockings,solid tights basic tigh sock nightwear woman w...,4
4,4,Shape Up 30 den 1p Tights,solid tights basic tigh sock nightwear woman a...,5
5,5,Support 40 den 1p Tights,solid tights basic tigh sock nightwear woman e...,6
6,6,200 den 1p Tights,solid tights basic tigh sock nightwear woman m...,7
7,7,SWEATSHIRT OC,pattern basic baby complement essential baby p...,8
8,8,Alice BANDEAU 2-p,stripesolid casual lingerie woman lingerie top...,9
9,9,Frugan longsleeve,solid jersey basic basic everyday woman neckli...,10


In [ ]:
#TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=21)
vectors = vectorizer.fit_transform(df['desc'])


In [ ]:
vectors = vectorizer.fit_transform(df['desc'].to_list())
vectors.shape

(10000, 21)

In [ ]:
x = df.product_id

In [ ]:
y = vectors.todense()

In [ ]:
#Add vectors to the dataset
import numpy as np
df['vectors'] = [y[i] for i in range(10000)]

In [ ]:
df.head()

,Unnamed: 0,product,desc,product_id,vectors
0,0,Strap top,solid jersey basic basic everyday woman should...,1,[[[[[0. 0. 0. 0. ...
1,1,Strap top (1),stripe jersey basic basic everyday woman shoul...,2,[[[[[0. 0. 0. 0. ...
2,2,OP T-shirt (Idro),solid clean lingerie woman lingerie visible mi...,3,[[[[[0.60479094 0. 0. 0. ...
3,3,20 den 1p Stockings,solid tights basic tigh sock nightwear woman w...,4,[[[[[0. 0. 0. 0. ...
4,4,Shape Up 30 den 1p Tights,solid tights basic tigh sock nightwear woman a...,5,[[[[[0. 0. 0. 0. ...


In [ ]:
#Calculate cosine similarity score for the vectors.
from sklearn.metrics.pairwise import cosine_similarity
cs =cosine_similarity(vectors)
cs.shape

(10000, 10000)

In [ ]:
def get_index_from_query(query):
    return df[df['product'] == query]['product_id'].values[0]
product_index = get_index_from_query("ROMY 2p")


In [ ]:
similar_items = list(enumerate(cs[product_index]))

In [ ]:
sorted_similar_items = sorted(similar_items, key=lambda x:x[1], reverse=True)

In [ ]:
#Get recommendations based on Tf-idf vectorization and cosine similarity score.
def get_product_from_index(index):
    return df[df['product_id'] == index]['product'].values[0]
i=0
for product in sorted_similar_items:
    print(get_product_from_index(product[0]))
    i=i+1
    if i>5:
        break

Strap top
Polly 3p minishaftless
ROMY 2p
Polly Top
SPEED Paprika LS
MIR cashmere/cotton trousers


In [ ]:
import pickle

In [ ]:
product_vector = {}
for key in df['product']:
  product_vector[key]=df[df['product']==key]['vectors'].values

In [ ]:
#product_vector

In [ ]:
with open('./product_vec.pkl', 'wb') as f:
  file = pickle.dump(product_vector, f)

In [ ]:
with open('./product_vec.pkl', 'rb') as f:
  file = pickle.load(f)

In [ ]:
#Perform Word Embedding
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
!wget https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g -O GoogleGoogleNews-vectors-negative300.bin

--2022-10-20 00:32:24--  https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g
Resolving drive.google.com (drive.google.com)... 172.217.218.101, 172.217.218.102, 172.217.218.138, ...
Connecting to drive.google.com (drive.google.com)|172.217.218.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘GoogleGoogleNews-vectors-negative300.bin’

GoogleGoogleNews-ve     [ <=>                ]  66.60K  --.-KB/s    in 0.08s   

2022-10-20 00:32:26 (874 KB/s) - ‘GoogleGoogleNews-vectors-negative300.bin’ saved [68199]



In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: glove.6B.100d.txt       


In [ ]:

import os
# Global parameters
#root folder
root_folder='.'
data_folder_name='data'
glove_filename='glove.6B.100d.txt'


# Variable for data directory
DATA_PATH = os.path.abspath(os.path.join(root_folder, data_folder_name))
glove_path = "/content/glove.6B.100d.txt"


In [ ]:
# glove2word2vec saves the Glove embeddings in the word2vec format 
from gensim.scripts.glove2word2vec import glove2word2vec
word2vec_output_file = glove_filename+'.word2vec'
glove2word2vec(glove_path, word2vec_output_file)

(400000, 100)

In [ ]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
word2vec_output_file = glove_filename+'.word2vec'
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)


In [ ]:
d_arr = df['desc'].to_numpy()
print(d_arr)


['solid jersey basic basic everyday woman shoulder top jersey strap narrow'
 'stripe jersey basic basic everyday woman shoulder top jersey strap narrow'
 'solid clean lingerie woman lingerie visible microfibre back padded comfort good support without bra hookandeye cup fastening shoulder seam provide bust greater lightly moulded tshirt underwired shape strap narrow adjustable'
 ...
 'solidmelangestructure jersey basic light men underwear elasticated drawstring pocket waist seam short cotton side piqué discreet'
 'solid jersey basic light men underwear sturdy roundnecked tshirt cotton jersey'
 'melangestripesolid jerseyjersey basic light men underwear sturdy roundnecked tshirt cotton jersey']


In [ ]:
#Maximum length of the vectors
final_vector =  {}
m = 0
for i in range (0,10000):

  word_vec = []
  for w in d_arr[i].split(" "):
    try:
      word_vec.append(sum(model.get_vector(w))/100)
    except:
      pass
  if m < len(word_vec):
    m = len(word_vec)
  final_vector[df['product'][i]] = word_vec
print(m)

67


In [ ]:
#Zero-padding of vectors
for key in final_vector.keys():
  if len(final_vector[key]) < m:
    r = m - len(final_vector[key])
    l = [0 for i in range(r)]
    final_vector[key].extend(l)

In [ ]:
# make the matrix with the words
words = final_vector.keys()
X = []
for w in words:
    X.append(final_vector[w])

from sklearn import cluster
from sklearn import metrics
kmeans = cluster.KMeans(n_clusters=500)
kmeans.fit(X)
print(kmeans)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for input data")
print (labels)
print ("Centroids data")
print (centroids)

KMeans(n_clusters=500)
Cluster id labels for input data
[141 141 492 ... 255 159 493]
Centroids data
[[-2.04795528e-04  7.27292312e-02 -6.62594573e-02 ... -8.47032947e-22
   8.47032947e-22 -2.11758237e-22]
 [ 4.61764736e-02 -5.30757024e-02  1.72123236e-02 ... -8.47032947e-22
   0.00000000e+00 -2.11758237e-22]
 [ 2.29394770e-02  2.25282005e-02  3.48359099e-02 ...  0.00000000e+00
   0.00000000e+00 -2.11758237e-22]
 ...
 [ 7.27292312e-02 -6.62594573e-02  8.22120105e-03 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-4.13567209e-03  8.44012352e-02  4.42389651e-02 ... -8.47032947e-22
   0.00000000e+00  0.00000000e+00]
 [-2.04366917e-02  2.63380876e-02  1.33334243e-02 ... -8.47032947e-22
   0.00000000e+00 -2.11758237e-22]]


In [ ]:
print(np.array(X).shape)


(10000, 67)


In [ ]:
#Find the cluster with least distance with the query vector
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
query ="Strap top"
v = np.array([final_vector[query]])
dist = 10000
for i in range(len(centroids)):
  d = cosine_similarity(v,[centroids[i]])
  if dist > d:
    
    dist = d
    label = labels[i]





In [ ]:
label

13

In [ ]:
#Getting the items present in the cluster.
ref = { i:X[i] for i in list(np.where(labels ==label)[0]) }
reff=[]
reff_key =[]
reff_key = list(ref.keys())
reff = list(ref.values())

In [ ]:
#distance between the query and the items present within the cluster
dist_list=[]
index_id =[]
v = np.array([final_vector[query]])
for i in range(len(reff)):
  d = cosine_similarity(v,[reff[i]])
  dist_list.append(list(d))
  index_id.append(reff_key[i])
  

In [ ]:
length = len(dist_list)

In [ ]:
length

33

In [ ]:
index_id

[19,
 20,
 23,
 221,
 316,
 433,
 434,
 436,
 441,
 445,
 446,
 450,
 550,
 681,
 935,
 1113,
 1114,
 1115,
 1204,
 1923,
 1924,
 2227,
 2290,
 3268,
 4666,
 4935,
 5162,
 5646,
 6012,
 6099,
 6326,
 7277,
 7813]

In [ ]:
print(dist_list)

[[array([0.44704779])], [array([0.44704779])], [array([0.34412666])], [array([0.50584998])], [array([0.3930144])], [array([0.41232548])], [array([0.4552634])], [array([0.4552634])], [array([0.47865251])], [array([0.3345321])], [array([0.3345321])], [array([0.43234416])], [array([0.72936586])], [array([0.43924916])], [array([0.1953162])], [array([0.44704779])], [array([0.44704779])], [array([0.44704779])], [array([0.40898064])], [array([0.33475749])], [array([0.33475749])], [array([0.03730507])], [array([0.38607547])], [array([0.46516655])], [array([0.32152456])], [array([-0.01086111])], [array([0.59817126])], [array([0.32653409])], [array([0.11369338])], [array([0.17145886])], [array([0.38751705])], [array([0.26545114])], [array([0.20388508])]]


In [ ]:
#Dictionary for index and respective difference of distance.
dictionary_list ={}
for i in range(len(dist_list)):
  dictionary_list[index_id[i]] =  dist_list[i]



In [ ]:
dictionary_list

{19: [array([0.44704779])],
 20: [array([0.44704779])],
 23: [array([0.34412666])],
 221: [array([0.50584998])],
 316: [array([0.3930144])],
 433: [array([0.41232548])],
 434: [array([0.4552634])],
 436: [array([0.4552634])],
 441: [array([0.47865251])],
 445: [array([0.3345321])],
 446: [array([0.3345321])],
 450: [array([0.43234416])],
 550: [array([0.72936586])],
 681: [array([0.43924916])],
 935: [array([0.1953162])],
 1113: [array([0.44704779])],
 1114: [array([0.44704779])],
 1115: [array([0.44704779])],
 1204: [array([0.40898064])],
 1923: [array([0.33475749])],
 1924: [array([0.33475749])],
 2227: [array([0.03730507])],
 2290: [array([0.38607547])],
 3268: [array([0.46516655])],
 4666: [array([0.32152456])],
 4935: [array([-0.01086111])],
 5162: [array([0.59817126])],
 5646: [array([0.32653409])],
 6012: [array([0.11369338])],
 6099: [array([0.17145886])],
 6326: [array([0.38751705])],
 7277: [array([0.26545114])],
 7813: [array([0.20388508])]}

In [ ]:
#Sorting in ascending order of distance value
dist_list.sort

<function list.sort(*, key=None, reverse=False)>

In [ ]:
#Top 5 vectors with least difference wrt distance
min_ten_dist =[]
for i in range(0,5):
  min_ten_dist.append(dist_list[i])
  

In [ ]:
min_ten_dist

[[array([0.44704779])],
 [array([0.44704779])],
 [array([0.34412666])],
 [array([0.50584998])],
 [array([0.3930144])]]

In [ ]:
#Product id of the vectors
top_five_index = [key for key,val in dictionary_list.items() if val in min_ten_dist[0:len(min_ten_dist)]]
top_five_index

[19, 20, 23, 221, 316, 1113, 1114, 1115]

In [ ]:
#Product names based on similarity
product_list =[]
for i in range(len(top_five_index)):
  product_list.append(df[df['product_id'] == top_five_index[i]]['product'].values[0])

In [ ]:
print(product_list)

['Dorian l/s basic', 'Tanktop body white 3PACK', '200 den 1p leggings', '2-p micro tights SG', 'Hype', 'Relaxed Skinny 5pkt', '6P Tanktopbody pack', '6PACK Tanktop Body']
